**Hash table** is another classic data structure that promises constant (**O(1)**) lookup and insertion time. <br>
You might have heard of the terms **"Dictionary"** or **Hash Map"**, but we will refer all of them as **"Hash Table"** in this chapter, as they are practically the same thing. 

In essence, **Hash Table** is a series of key-value pairs, where the key is used to find the position where the data is stored, and value is the one that contains the actual data.<br> **Hashing** or **Hash Function** is the key technique used that given any arbitrary key(can be string, int, double, or even objects), computes a corresponding integer index value that tells the computer where to retrive the data.

In Python, there is one such built-in data structure called **dictionary**, and you can initilize a dict by calling **dict()**. But here we will implement our own **hash table** to understand the ins and outs of this data structure.

In [1]:
# A very straight forward hash function for string keys
def hash(str_key):
    return sum([ord(char) for char in str_key])

Assuming our key is of type string (which is likely in many cases), in order to convert it into a integer, the above function use the built-in **ord()** function to convert each character into its corresponding Unicode code and sum them up to get a single positive integer.<br> This resulting integer will later be used as the index value to insert and retrive data.

In [2]:
def test_hash(hash_func, str_key):
    # f-string is a formatted string
    return f'Hash value of "{str_key}" is {hash_func(str_key)}.' 
print(test_hash(hash, 'hello')) 
print(test_hash(hash, 'world'))
print(test_hash(hash, 'hello world'))
print(test_hash(hash, 'gello xorld'))

Hash value of "hello" is 532.
Hash value of "world" is 552.
Hash value of "hello world" is 1116.
Hash value of "gello xorld" is 1116.


Some test code. Note how the strings "hello world" and "gello xorld" have the indentical hash values, which is called a **collision**.<br>Collision is no good because if forces us to put two different items into the same location (since the hash value indicates where they are stored), which will slow down loopup in return.<br>Ideally, we want to **minimize the number of collisions** (Naively, we could possibly create a function which outputs a unique integer for each possible string, but as you will see later, it will waste a lot of memory).

In [3]:
# A better version of the hash function
def better_hash(str_key):
    return sum([ord(char) * (index + 1) for (index, char) in enumerate(str_key)])

In [4]:
print(test_hash(better_hash, 'hello')) 
print(test_hash(better_hash, 'world'))
print(test_hash(better_hash, 'hello world'))
print(test_hash(better_hash, 'gello xorld'))

Hash value of "hello" is 1617.
Hash value of "world" is 1615.
Hash value of "hello world" is 6736.
Hash value of "gello xorld" is 6742.


In order to address the issue we mentioned and to reduce the number of collisions, we propose a better version of the hash function, which takes the orders of the characters into account.<br> For each character, we take its Unicode code as usual, and **multiply it by the character's index in the string**. Finally we sum everything up to get a single result.<br> As you can see from the test results, "hello world" and "gello xorld" no longer have the same hash values. 

**Note**: Even though this function is slightly better, it is still far from a **perfect hash function** where the number of collisions is minimized.<br>In real world, this problem can be an area of study, and so far some great functions which are backed by mathemitical theories are commonly used to implement efficient hash tables (Do some research by yourself if you are interested).

In [5]:
class HashTable:
    def __init__(self, table_size = 256):
        self.table_size = table_size
        self.data = [None] * table_size
        
    # The hash function; 
    # this function can be anything you want, 
    # as long as it returns a integer that is within the range(0, talbe_size)
    def _hash(self, str_key):
        v = sum([ord(char) * (index + 1) for (index, char) in enumerate(str_key)])
        return v % self.table_size
        
    def add(self, key, value):
        hash_value = self._hash(key)
        if self.data[hash_value] is None:
            self.data[hash_value] = [(key, value)]
        else:
            self.data[hash_value].append((key, value))
            
    def get(self, key):
        hash_value = self._hash(key)
        if self.data[hash_value] is None:
            return None
        for stored_key, value in self.data[hash_value]:
            if key == stored_key:
                return value
        return None

Now that the core of our hash table is ready, we define our class for hash table.<br> As you can see, we use a list with fixed size (to reduce the memory usage) to store all the key value pairs.<br>**_hash** function (the understore before hash is a common convention in Python which basically means "private") is the hash function used earilier, with the addition of a **(% table_size)** in the end to fix the value into the defined range. <br><br>Now consider if we do not try to fix the range, a raw hash value 6742 (as in the case of "gello world" seen earlier) will require the list to have a size of at least 6742, where a lot of positions might just be wasted momory.<br><br> The **add** function uses the key and the hash function to get the index where the data should be inserted, and insert it accordingly.<br> Similarly, The **get** function uses the key to fetch the index, and get the wanted value.<br><br> **Note**: We always need to address the issue of collisions, and in this case, the list is actually a **2-dimentional list** so multiple values can be inserted into a single position. But this also adds a few extra lines to **add** and **get** as we need to traverse the sublist in order to get what we want.    

In [6]:
phone_book = HashTable(512)
phone_book.add("tom", "123-467-0000")
phone_book.add("jack", "777-888-9394")
phone_book.add("mary", "666-0303-2222")
print(f"tom\'s phone number is {phone_book.get('tom')}.")
print(f"mary\'s phone number is {phone_book.get('mary')}.")
print(f"jerry\'s phone number is {phone_book.get('jerry')}.")

tom's phone number is 123-467-0000.
mary's phone number is 666-0303-2222.
jerry's phone number is None.


Some test code, where the name is used as the key and phone number is the value in this "phone_book" hash table.<br>A query about "jerry" returns None because his number was never recorded. 

At this point, this hash table is fully functional, maybe except a delete(key) functionality, but that should be easily doable as it is very similar to our get function.<br>It's just that instead of returning the value when the pair is found, we delete the pair from the sublist (Use index and pop() on the sublist).<br><br> However, if you have ever used Python's built-in dictionary, you might not like our syntax, as we have to **explicitly call add and get** on the hash table, where in the case of Python dict, you can almost use it just like a normal list. So let's try to clone that.

In [7]:
class HashTable:
    def __init__(self, table_size = 256):
        self.table_size = table_size
        self.data = [None] * table_size
        
    # The hash function; 
    # this function can be anything you want, 
    # as long as it returns a integer that is within the range(0, talbe_size)
    def _hash(self, str_key):
        v = sum([ord(char) * (index + 1) for (index, char) in enumerate(str_key)])
        return v % self.table_size
        
    def add(self, key, value):
        hash_value = self._hash(key)
        if self.data[hash_value] is None:
            self.data[hash_value] = [(key, value)]
        else:
            self.data[hash_value].append((key, value))
            
    def get(self, key):
        hash_value = self._hash(key)
        if self.data[hash_value] is None:
            return None
        for stored_key, value in self.data[hash_value]:
            if key == stored_key:
                return value
        return None
    
    def __setitem__(self, key, value):
        self.add(key, value)
    
    def __getitem__(self, key):
        return self.get(key)

In [8]:
my_final_report = HashTable()
my_final_report["math"] = 90
my_final_report["advanced programming"] = 85
my_final_report["operating system"] = 75
my_final_report["principles of programming language"] = 50  # Opps
print(f'I got {my_final_report["math"]} for my math final.')
print(f'I got {my_final_report["principles of programming language"]} for my POPL final.')
print(f'My grade for history is {my_final_report["history"]} because I never took it.')

I got 90 for my math final.
I got 50 for my POPL final.
My grade for history is None because I never took it.


As you can see, Python actually makes this pretty easy (which is amazing) by defining these two functions **__setitem__** and **__getitem__**. I hope you enjoy it as much as I did.

We covered quite a lot to unveil the myth of **Hash Table**. Since you understand how it works now, be happy to use Python's built-in **dictionary** to solve problems when applicable (Usually for the constant lookup time complexity, very handy in CS interviews).<br><br> 
__Time Complexities__<br>
<ul>
    <li><b>add</b>: O(1) on average, assuming the hash function is performing reasonably well</li>
    <li><b>get</b>: O(1) on average</li>
    <li><b>delete</b>: O(1) on average</li>
</ul>